<a href="https://colab.research.google.com/github/Hadiasemi/Data301/blob/main/Copy_of_11_3_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 11.3 Web Scraping

**HTML**, which stands for "hypertext markup language", is an XML-like language for specifying the appearance of web pages. Each tag in HTML corresponds to a specific page element. For example:

- `<img>` specifies an image. The path to the image file is specified in the `src=` attribute.
- `<a>` specifies a hyperlink. The text enclosed between `<a>` and `</a>` is the text of the link that appears, while the URL is specified in the `href=` attribute of the tag.
- `<table>` specifies a table. The rows of the table are specified by `<tr>` tags nested inside the `<table>` tag, while the cells in each row are specified by `<td>` tags nested inside each `<tr>` tag.

Our goal in this section is not to teach you HTML to make a web page. You will learn just enough HTML to be able to scrape data programmatically from a web page.

# Inspecting HTML Source Code

Suppose we want to scrape faculty information from the [Cal Poly Statistics Department directory](https://statistics.calpoly.edu/content/directory) (`https://statistics.calpoly.edu/content/directory`). Once we have identified a web page that we want to scrape, the next step is to study the HTML source code. All web browsers have a "View Source" or "Page Source" feature that will display the HTML source of a web page. 

Visit the web page above, and view the HTML source of that page. (You may have to search online to figure out how to view the page source in your favorite browser.) Scroll down until you find the HTML code for the table containing information about the name, office, phone, e-mail, and office hours of the faculty members.

Notice how difficult it can be to find a page element in the HTML source. Many browsers allow you to right-click on a page element and jump to the part of the HTML source corresponding to that element.

# Web Scraping Using BeautifulSoup

`BeautifulSoup` is a Python library that makes it easy to navigate an HTML document. Like with `lxml`, we can query tags by name or attribute, and we can narrow our search to the ancestors and descendants of specific tags. In fact, it is possible to use `lxml` with HTML documents, but many web sites have malformed HTML, and `lxml` is not very forgiving. `BeautifulSoup` handles malformed HTML more gracefully and is thus the library of choice.

First, we issue an HTTP request to the URL to get the HTML source code.

In [1]:
import requests
response = requests.get("https://statistics.calpoly.edu/content/directory")

The HTML source is stored in the `.content` attribute of the response object. We pass this HTML source into `BeautifulSoup` to obtain a tree-like representation of the HTML document.

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

Now we can search for tags within this HTML document, using tags like `.find_all()`. For example, we can find all tables on this page.

In [3]:
tables = soup.find_all("table")
len(tables)

2

As a visual inspection of [the web page](https://statistics.calpoly.edu/content/StatisticsDirectory%26Office%20Hours) would confirm, there are 2 tables on the page, and we are interested in the second one.

In [4]:
table = tables[1]
table

<table cellspacing="0" class="directory backwhite" height="2667" summary="COSAM Dean's Office Staff" width="596"><tbody><tr><th scope="col" width="18%">
<p>Name</p>
</th>
<th scope="col" width="8%">Office</th>
<th scope="col" width="10%">Phone (805)</th>
<th scope="col" width="20%">Email</th>
<th scope="col" width="20%">
<p>Office HoURS </p>
</th>
</tr><tr><td><a href="/Kelly-Bodwin"><strong>Kelly Bodwin</strong></a></td>
<td><a href="https://maps.calpoly.edu/">25-106</a></td>
<td>756-2450</td>
<td><a href="mailto:kbodwin@calpoly.edu">kbodwin@calpoly.edu</a></td>
<td>
<p><strong>TWR 2:30-3:30pm</strong></p>
<p><strong>W 2:30-4:30pm</strong></p>
</td>
</tr><tr><td><a href="/Matt-Carlton"><strong>Matt Carlton</strong></a></td>
<td><a href="https://maps.calpoly.edu/">25-101</a></td>
<td>756-7076</td>
<td><a href="mailto:rmlau@calpoly.edu">mcarlton@calpoly.edu</a></td>
<td>
<p><strong>M 1:10-2pm</strong></p>
<p><strong>WR 2:10-3pm</strong></p>
<p><strong>F 4:10-6pm Zoom</strong></p>
<p>See

There is one faculty member per row, except for the first row, which is the header. We iterate over all rows except for the first, extracting information about each faculty to append to `rows`, which we will eventually turn into a `DataFrame`. As you read the code below, refer to the HTML source above, so that you understand what each line is doing. In particular, ask yourself the following questions:

- `cells[0]` represents a `<td>` tag. Why do we need to call `.find("strong")` within this tag to get the name of the faculty member?
- For the most part, `link` is a hyperlink whose text is the faculty's office number. But for some faculty, `link` is `None`. For which faculty is this the case and why?

You are encouraged to add `print()` statements inside the `for` loop to check your understanding of each line of code.

In [8]:
rows = []

for faculty in table.find_all("tr")[1:]:
    # Get all the cells in the row.
    cells = faculty.find_all("td")
    
    # The information we need is the text between tags.
    name_tag = cells[0].find("strong") or cells[0]
    name = name_tag.text
    
    link = cells[1].find("a")
    office = cells[1].text if link is None else link.text
    
    email_tag = cells[3].find("a") or cells[3]
    email = email_tag.text
    
    # Append this data.
    rows.append({
        "name": name,
        "office": office,
        "email": email
    })

In the code above, observe that `.find_all()` returns a list with all matching tags, while `.find()` returns only the first matching tag. If no matching tags are found, then `.find_all()` will return an empty list `[]`, while `.find()` will return `None`.

Finally, we turn `rows` into a `DataFrame`.

In [6]:
import pandas as pd
pd.DataFrame(rows)

,name,office,email
0,Kelly Bodwin,25-106,kbodwin@calpoly.edu
1,Matt Carlton,25-101,mcarlton@calpoly.edu
2,Beth Chance,25-235,bchance@calpoly.edu
3,Nianpin Cheng,92M-A103,ncheng@calpoly.edu
4,Olga Dekhtyar,25-225,odekhtya@calpoly.edu
5,Jimmy Doi,25-108,jdoi@calpoly.edu
6,Samuel Frame,25-121,sframe@calpoly.edu
7,Hunter Glanz,25-111,hglanz@calpoly.edu
8,Giri Gopalan,25-103,ggopalan@calpoly.edu
9,Billie-Jo Grant,25-221,bgrant02@calpoly.edu


Now this data is ready for further processing.

# Ethical Enlightenment: `robots.txt`

Web robots are crawling the web all the time. A website may want to restrict the robots from crawling specific pages. One reason is financial: each visit to a web page, by a human or a robot, costs the website money, and the website may prefer to save their limited bandwidth for human visitors. Another reason is privacy: a website may not want a search engine to preserve a snapshot of a page for all eternity.

To specify what a web robot is and isn't allowed to crawl, most websites will place a text file named `robots.txt` in the top-level directory of the web server. For example, the Statistics department web page has a `robots.txt` file: https://statistics.calpoly.edu/robots.txt

The format of the `robots.txt` file should be self-explanatory, but you can read a full specification of the standard here: http://www.robotstxt.org/robotstxt.html. As you scrape websites using your web robot, always check the `robots.txt` file first, to make sure that you are respecting the wishes of the website owner.

# Exercises

1\. The [Cal Poly course catalog](http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory) (`http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory`) contains a list of courses offered by the Statistics department. Scrape this website to obtain a `DataFrame`, where the unit of observation is a course. Use this `DataFrame` to answer the following questions: 

- How many 300-level courses does the Statistics department offer?
-How many three-unit courses are offered?

In [54]:
response = requests.get("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

data = []
soup = BeautifulSoup(response.content, "html.parser")

for clas in soup.find_all("div", class_="courseblock"):
  cells = clas.find_all('p', class_="courseblocktitle")
  title = cells[0].find("strong").text.split('.')
  level = title[0].strip().split()
  unit = title[2].strip().split(" ")
  # print(level[0])
  data.append({
      "class": level[0],
      "num": int(level[1]),
      "units": unit[0]
  })

df = pd.DataFrame(data)

In [55]:
df

,class,num,units
0,DATA,301,4
1,DATA,401,3
2,DATA,402,3
3,DATA,403,1
4,DATA,441,2
...,...,...,...
68,STAT,551,4
69,STAT,566,2
70,STAT,570,1-4
71,STAT,590,1


In [57]:

df[(df["class"] == "STAT") & (df['num'] > 300) & (df['num'] < 400)]

,class,num,units
17,STAT,301,4
18,STAT,302,4
19,STAT,305,4
20,STAT,312,4
21,STAT,313,4
22,STAT,314,4
23,STAT,321,4
24,STAT,323,4
25,STAT,324,4
26,STAT,330,4
